In [14]:
from pydub import AudioSegment
from pydub.generators import Sine
import math
import musicpy as mp

color_to_freq_map = {
    '#FF0000': 261.63,  # C4
    '#FFA500': 392.00,  # G4
    '#FFFF00': 293.66,  # D4
    '#008000': 440.00,  # A4
    '#ADD8E6': 329.63,  # E4
    '#87CEEB': 493.88,  # B4
    '#0000FF': 369.99,  # F#4
    '#EE82EE': 277.18,  # Db4
    '#800080': 415.30,  # Ab4
    '#B0C4DE': 311.13,  # Eb4
    '#5F9EA0': 466.16,  # Bb4
    '#8B0000': 349.23,  # F4
}

color_chord_map = {
    '#FF0000': 'C',   # C or chord for red
    '#FFA500': 'G',   # G or chord for orange
    '#FFFF00': 'D',   # D or chord for yellow
    '#008000': 'A',   # A minor chord for green
    '#ADD8E6': 'E',   # E or chord for light blue
    '#87CEEB': 'B',   # B or chord for sky blue
    '#0000FF': 'F#',  # F# major chord for blue
    '#EE82EE': 'Db',  # Db major chord for violet
    '#800080': 'Ab',  # Ab major chord for purple
    '#B0C4DE': 'Eb',  # Eb major chord for steel blue
    '#5F9EA0': 'Bb',  # Bb major chord for cadet blue
    '#8B0000': 'F',   # F major chord for dark red
    # ... add all necessary mappings
}


def color_to_chord_map(color):
    # Define your color to chord mapping here
    approx_color = closest_color_from_hex(color, color_chord_map)
    return color_chord_map.get(approx_color, None)


def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb):
    """Convert an RGB color to hexadecimal format."""
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

def color_distance(rgb1, rgb2):
    """Calculate the Euclidean distance between two RGB colors."""
    return math.sqrt(sum((c1 - c2) ** 2 for c1, c2 in zip(rgb1, rgb2)))

def closest_color_from_hex(color, color_map):
    rbg_color = hex_to_rgb(color)
    closest_colors = sorted(color_map.keys(), key=lambda color: color_distance(rbg_color, hex_to_rgb(color)))
    return closest_colors[0]

def closest_color(rgb, color_map):
    """Find the closest color in the color_map to the given RGB color."""
    closest_colors = sorted(color_map.keys(), key=lambda color: color_distance(rgb, hex_to_rgb(color)))
    return closest_colors[0]

def freq_to_note_name(freq):
    # This is a simplification, in reality, you would need to account for tuning and the exact frequencies of notes
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    return notes[round(math.log2(freq / 440.0) * 12) % 12]

def colors_to_scale(main_colors, color_to_freq_map):

    notes = [freq_to_note_name(color_to_freq_map[closest_color(hex_to_rgb(color), color_to_freq_map)]) for color in main_colors]
    chord_obj = mp.chord(notes[:2])
    scales = mp.alg.detect(chord_obj, show_degree=True)
    return scales


In [15]:
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans

def load_and_cluster(path, n_clusters, display=False):
    image = Image.open(path)
    x, y = image.size

    h = 150 / float(y)

    redim = min(x, h)

    new_h = int(y * redim)

    image = image.resize((x, new_h), Image.LANCZOS)

    x, y = image.size

    if image.mode != 'RGB':
        image = image.convert('RGB')

    pixels = list(image.getdata())
    pixels_list = [pixels[i * x:(i + 1) * x] for i in range(y)]
    image_array = np.array(image)
    pixels = image_array.reshape(-1, 3)

    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(pixels)
    labels = kmeans.labels_

    for i in range(y):
        for j in range(x):
            pixel_index = i * x + j
            pixels_list[i][j] = labels[pixel_index]
    centers = [rgb_to_hex(i) for i in kmeans.cluster_centers_.astype(int)]
    return centers, pixels_list

In [16]:
def image_to_music(main_colors, pixels, note_duration=1000, min_length=0.05):
    min_note_length = note_duration * min_length

    # Create an empty musicpy piece
    chords = []

    for row in pixels:
        row_length = len(row)
        current_color = main_colors[row[0]]
        current_count = 1
        note_groups = []

        # Iterate over the row to group colors
        for idx in row[1:]:
            hex_color = main_colors[idx]
            if hex_color == current_color:
                current_count += 1
            else:
                note_groups.append((current_color, current_count))
                current_color = hex_color
                current_count = 1
        note_groups.append((current_color, current_count))

        total_above_min = sum(count for color, count in note_groups if count / row_length >= min_length)

        # Generate chords with respective durations
        for hex_color, count in note_groups:
            proportion = count / row_length
            note_time = min_note_length if proportion < min_length else note_duration * (count / total_above_min)

            # Convert the color to a chord type
            chord_type = color_to_chord_map(hex_color)
            root_note = color_to_chord_map(hex_color)  # This assumes the root note of the chord is determined from the color

            # Create the chord object with the desired duration
            current_chord = mp.get_chord(root_note, current_chord_type='maj', duration=note_time/1000)

            # Add the chord to the piece
            chords.append(current_chord)
    piece = mp.piece(tracks=chords, channels=1)
    # Play the piece
    mp.write(piece, bpm=60*1000/note_duration)


In [17]:
main_colors, pixels_list = load_and_cluster("Flag_of_Sudan.png", 3)
scales = colors_to_scale(main_colors, color_to_freq_map)
print(scales)
print(main_colors)

C with perfect fifth
['#00290e', '#fefefe', '#d11033']


In [22]:
image_to_music(main_colors, pixels_list, min_length=0.125)

TypeError: 'int' object is not subscriptable

In [19]:
def generate_pattern_number(n, seed=None):
    """
    Generates a pattern number with N digits, each digit is between 1 and 6.

    Args:
    - n (int): The number of digits in the pattern number.
    - seed (int): The seed for the random number generator.

    Returns:
    - int: A pattern number with N digits.
    """
    if seed is not None:
        random.seed(seed)
    pattern_number = 0
    for _ in range(n):
        digit = random.randint(1, 6)
        pattern_number = pattern_number * 10 + digit
    return pattern_number

In [20]:
def generate_gaussian_between(b1, b2, mean, std_dev, seed=None):
    """
    Generates a random Gaussian variable between b1 and b2 with a specified mean and standard deviation.

    Args:
    - b1 (float): The lower bound of the generated number.
    - b2 (float): The upper bound of the generated number.
    - mean (float): The mean of the Gaussian distribution.
    - std_dev (float): The standard deviation of the Gaussian distribution.
    - seed (int, optional): The seed for the random number generator.

    Returns:
    - float: A random Gaussian variable within the specified bounds.
    """
    if seed is not None:
        np.random.seed(seed)

    while True:
        number = np.random.normal(mean, std_dev)
        if b1 <= number <= b2:
            return number

In [21]:
def weighted_mode_selection(mean, std_dev, seed=None):
    """
    Selects an element from the mode_list with a higher likelihood of selecting 'major' or 'minor'.

    Args:
    - mean (float): The mean of the Gaussian distribution.
    - std_dev (float): The standard deviation of the Gaussian distribution.
    - seed (int, optional): The seed for the random number generator.

    Returns:
    - str: A mode selected from the mode_list.
    """
    if seed is not None:
        np.random.seed(seed)

    # Assuming 'major' and 'minor' should have higher probabilities
    # Calculate the index bounds based on the list size
    b1, b2 = 0, len(mode_list) - 1

    # Generate a Gaussian distributed index
    while True:
        index = int(np.round(np.random.normal(mean, std_dev)))
        if b1 <= index <= b2:
            return mode_list[index]


In [ ]:
def generate_chords(color):
    mode_list = [
        'dorian',
        'phrygian',
        'minor',
        'major',
        'lydian',
        'mixolydian',
        'locrian'
    ]

    chord_choice = color_to_chord_map(color)
    mode_choice = weighted_mode_selection(3, 1)
    pattern_number = generate_pattern_number(1)

    rythmic_division_pow = random.randint(0, 5)
    rythmic_division = 2**rythmic_division_pow

    tempo_division_pow = random.randint(0, 5)
    tempo = 2**tempo_division_pow

    # print(f"Chord:{chord_choice}\nMode:{mode_choice}")
    # print(f"Pattern number:{pattern_number}\nRythmic div:1/{rythmic_division}")
    # print(f"Tempo:1/{tempo}\n")

    chords = S(f'{chord_choice} {mode_choice}') % (pattern_number, 1/4, 1/rythmic_division)



In [6]:
from musicpy.daw import *
import random
import numpy as np

chord_list = [
    'C',   # C or chord for red
    'G',   # G or chord for orange
    'D',   # D or chord for yellow
    'A',   # A minor chord for green
    'E',   # E or chord for light blue
    'B',   # B or chord for sky blue
    'F#',  # F# major chord for blue
    'Db',  # Db major chord for violet
    'Ab',  # Ab major chord for purple
    'Eb',  # Eb major chord for steel blue
    'Bb',  # Bb major chord for cadet blue
    'F',   # F major chord for dark red
    # ... add all necessary mappings
]

mode_list = [
    'major',
    'minor',
    'dorian',
    'phrygian',
    'lydian',
    'mixolydian',
    'locrian'
]

chord_choice = random.choice(chord_list)
mode_choice = weighted_mode_selection(0, 0.5)
pattern_number = generate_pattern_number(4)
rythmic_division_pow = random.randint(0, 5)
rythmic_division = 2**rythmic_division_pow
tempo_division_pow = random.randint(0, 5)
tempo = 2**tempo_division_pow
print(f"Chord:{chord_choice}\nMode:{mode_choice}")
print(f"Pattern number:{pattern_number}\nRythmic div:1/{rythmic_division}")
print(f"Tempo:1/{tempo}\n")

chords = S(f'{chord_choice} {mode_choice}') % (pattern_number, 1/4, 1/rythmic_division)

play(chords, bpm=100)


Chord:Eb
Mode:major
Pattern number:6311
Rythmic div:1/1
Tempo:1/2



In [7]:
stopall()